<a href="https://colab.research.google.com/github/lazygoonlife/cv-project--basic-ASL-detector-/blob/main/ASL_Identifier_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
import sys
import subprocess
import matplotlib.pyplot as plt
from PIL import Image

# ==========================================
# 1. ENVIRONMENT SETUP & FIXES
# ==========================================
def setup_environment():
    """
    Installs specific versions of OpenCV and Numpy to resolve the
    '(-5:Bad argument) in function imdecode' error common in Colab/Linux environments.
    """
    print("🔧 Setting up environment and fixing dependencies...")
    packages = [
        "opencv-python-headless<4.11", # Fix for cv2 error
        "numpy<2.0",                   # Fix for binary compatibility
        "ultralytics",
        "kagglehub",
        "torch",
        "torchvision",
        "matplotlib",
        "scipy"                        # Added scipy explicitly to ensure compatibility
    ]

    for package in packages:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

    print("✅ Environment setup complete. Note: If in Colab, you may need to restart the runtime once if errors persist.")

# Run setup immediately
setup_environment()

# Imports after installation
import kagglehub
import torch
from torch.utils.data import Dataset, random_split
from torchvision import transforms
from ultralytics import YOLO

# ==========================================
# 2. DATA ACQUISITION & PROCESSING
# ==========================================
class ASLDataset(Dataset):
    """Custom Dataset class to handle loading and indexing of ASL images."""
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.image_paths = []
        self.labels = []

        # Filter for valid class directories (0-9, a-z)
        self.classes = sorted([
            d for d in os.listdir(root_dir)
            if os.path.isdir(os.path.join(root_dir, d)) and len(d) == 1 and d.isalnum()
        ])
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}

        for class_name in self.classes:
            class_path = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                if os.path.isfile(img_path):
                    self.image_paths.append(img_path)
                    self.labels.append(self.class_to_idx[class_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        return self.image_paths[idx], self.labels[idx]

def prepare_data():
    """Downloads dataset and creates YOLO-compatible directory structure via symlinks."""
    print("\n📦 Downloading and Preparing Data...")

    # Download
    dataset_path = kagglehub.dataset_download("ayuraj/asl-dataset")
    base_path = os.path.join(dataset_path, 'asl_dataset')

    # Initialize Dataset
    full_dataset = ASLDataset(root_dir=base_path)

    # Split
    train_size = int(0.7 * len(full_dataset))
    val_size = int(0.15 * len(full_dataset))
    test_size = len(full_dataset) - train_size - val_size
    train_set, val_set, test_set = random_split(full_dataset, [train_size, val_size, test_size])

    # Create YOLO Structure
    yolo_root = './yolo_classify_data'
    for split_name, split_data in [('train', train_set), ('val', val_set), ('test', test_set)]:
        split_dir = os.path.join(yolo_root, split_name)
        os.makedirs(split_dir, exist_ok=True)

        for idx in split_data.indices:
            img_path = full_dataset.image_paths[idx]
            label_idx = full_dataset.labels[idx]
            class_name = full_dataset.classes[label_idx]

            target_dir = os.path.join(split_dir, class_name)
            os.makedirs(target_dir, exist_ok=True)

            # Symlink
            link_path = os.path.join(target_dir, os.path.basename(img_path))
            if not os.path.exists(link_path):
                os.symlink(img_path, link_path)

    print(f"✅ Data prepared at {yolo_root}")
    return yolo_root, full_dataset

# ==========================================
# 3. MODEL TRAINING
# ==========================================
def train_model(data_root):
    """Trains the YOLOv8 classification model."""
    print("\n🚀 Starting YOLOv8 Training...")

    model = YOLO('yolov8n-cls.pt') # Load pretrained classification model

    results = model.train(
        data=data_root,
        epochs=5,
        imgsz=224,
        batch=16,
        project='runs/classify',
        name='asl_model',
        exist_ok=True # Overwrite existing run
    )
    print("✅ Training Complete.")
    return model

# ==========================================
# 4. PROOF OF WORK (Inference & Viz)
# ==========================================
def prove_it_works(full_dataset):
    """
    Runs robust tests to prove the model works:
    1. Single Image Inference (fixing the cv2 error)
    2. Visualization of Training Metrics
    3. Confusion Matrix display
    """
    print("\n🔎 Initiating Proof of Work Protocols...")

    # Path to best weights
    weights_path = 'runs/classify/asl_model/weights/best.pt'
    model = YOLO(weights_path)

    # --- TEST 1: Robust Inference ---
    # Pick a specific sample image we know exists
    sample_img_path = full_dataset.image_paths[0]
    true_label = full_dataset.classes[full_dataset.labels[0]]

    print(f"\n🧪 Test 1: Predicting on sample image...")
    print(f"   Image: {sample_img_path}")
    print(f"   True Label: {true_label}")

    # Run inference
    results = model(sample_img_path)

    # Extract results
    top1_idx = results[0].probs.top1
    top1_conf = results[0].probs.top1conf.item()
    pred_label = results[0].names[top1_idx]

    print(f"   ✅ Prediction: {pred_label}")
    print(f"   📊 Confidence: {top1_conf:.2%}")

    if pred_label == true_label:
        print("   🎉 SUCCESS: Prediction matches ground truth!")
    else:
        print("   ⚠️ NOTE: Prediction differs (Check training epochs/data balance).")

    # --- TEST 2: Visualize Metrics ---
    print("\n📈 Test 2: Visualizing Training Results...")
    results_img_path = 'runs/classify/asl_model/results.png'

    if os.path.exists(results_img_path):
        img = Image.open(results_img_path)
        plt.figure(figsize=(10, 6))
        plt.imshow(img)
        plt.axis('off')
        plt.title("Training Loss & Accuracy Curves")
        plt.show()
        print("   ✅ Training metrics plotted.")
    else:
        print("   ⚠️ results.png not found (Training might be too short).")

    # --- TEST 3: Confusion Matrix ---
    print("\n🧩 Test 3: Checking Confusion Matrix...")
    cm_img_path = 'runs/classify/asl_model/confusion_matrix.png'

    if os.path.exists(cm_img_path):
        img = Image.open(cm_img_path)
        plt.figure(figsize=(8, 8))
        plt.imshow(img)
        plt.axis('off')
        plt.title("Confusion Matrix")
        plt.show()
        print("   ✅ Confusion Matrix plotted.")
    else:
        # Generate it manually if YOLO didn't save it automatically during val
        print("   ⚠️ Matrix image not found, running validation to generate stats...")
        model.val() # This usually generates the matrix
        print("   ✅ Validation run complete.")

# ==========================================
# MAIN EXECUTION FLOW
# ==========================================
if __name__ == "__main__":
    # 1. Prepare Data
    yolo_path, dataset_obj = prepare_data()

    # 2. Train
    trained_model = train_model(yolo_path)

    # 3. Prove
    prove_it_works(dataset_obj)


🔧 Setting up environment and fixing dependencies...
✅ Environment setup complete. Note: If in Colab, you may need to restart the runtime once if errors persist.
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

📦 Downloading and Preparing Data...


100%|██████████| 56.9M/56.9M [00:00<00:00, 178MB/s]

Extracting files...


✅ Data prepared at ./yolo_classify_data

🚀 Starting YOLOv8 Training...
Ultralytics 8.3.232 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./yolo_classify_data, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=asl_model, nbs=64, nms=False, opset=None, optimize=False, optimizer=aut

ImportError: cannot import name '_center' from 'numpy._core.umath' (/usr/local/lib/python3.12/dist-packages/numpy/_core/umath.py)